In [15]:
import numpy as np
import scipy
import matplotlib.pyplot as plt
from scipy.integrate import simpson
from scipy.optimize import least_squares

def SIR(y, t, beta, gamma):
    """
    :param y: initial condition
    :param t: time
    :param beta: para
    :param gamma: para
    :return: the SIR model ODEs
    """
    S, I, R = y
    N = S + I + R
    dS = - beta * S * I / N
    dI = beta * S * I / N - gamma * I
    dR = gamma * I
    return [dS, dI, dR]

In [16]:
# initial condition
S0 = 9999
I0 = 1
R0 = 0
y = S0, I0, R0

# true parameters
beta = 0.0001
gamma = 0.01

# time
t = np.linspace(0, 500, 1000)

# simulate SIR
solution = scipy.integrate.odeint(SIR, [S0, I0, R0], t, args=(beta, gamma))
S, I, R = solution.T

In [17]:
I_clean = I
I_noisy = I + np.random.normal(0, 1, size=I.shape)

# estimate the integral using the data points
I_data = I_noisy # we could change this to I clean freely

I_int = np.array([simpson(I_data[:i+1], t[:i+1]) for i in range(len(t))])
I_int2 = np.array([simpson(I_data[:i+1]**2, t[:i+1]) for i in range(len(t))])
int_double = 1/2 * (I_int ** 2)

In [18]:
def residual(paras, I_data, I_int, I_int2, int_double, I0):
    beta, gamma, S0 = paras
    I_hat = I0 + ((beta * S0 + beta * I0) - gamma) * I_int - beta * I_int2 - beta * gamma * int_double
    return I_data - I_hat

# initial guess value of beta and gamma
beta0 = 0.01
gamma0 = 0.05
S0_initial = 100
x0 = [beta0, gamma0, S0_initial]

res = least_squares(residual, x0, args=(I_data, I_int, I_int2, int_double, I0))
print(f"Estimated beta: {res.x[0]} --- real beta {beta}"
      f"\nEstimated gamma: {res.x[1]} --- real gamma {gamma}"
      f"\nEstimated S0: {res.x[2]} -- real S0 {S0}")

Estimated beta: 0.00015771287503956385 --- real beta 0.0001
Estimated gamma: 0.38311711852552416 --- real gamma 0.01
Estimated S0: 2386.809686062401 -- real S0 9999
